In [87]:
import numpy as np
import pandas as pd
import random
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, mean_squared_error
import sklearn.cross_validation as cv
from sklearn.cross_validation import KFold, train_test_split
from sklearn import preprocessing
import sklearn.metrics
from sklearn.grid_search import GridSearchCV
import time
from sklearn import ensemble
min_max_scaler = preprocessing.MinMaxScaler()

In [88]:
#####################
# Read train and test
#####################
train = pd.read_csv("/Users/boulenge/Desktop/Projects/Project4 - ML/train.csv", sep=',',\
                        encoding='utf-8')#, header=None, skiprows = 1)
train = train.set_index("ID")
test = pd.read_csv("/Users/boulenge/Desktop/Projects/Project4 - ML/test.csv", sep=',',\
                        encoding='utf-8')#, header=None, skiprows = 1)
test = test.set_index("ID")

In [89]:
#############################
# Remove the similar features
#############################
Full_train = train.iloc[:, :369]
temp = Full_train
ind_same = {}
for i in range(len(temp.columns)):
    tp = temp.drop(temp.columns[i], inplace=False, axis=1)
    for j in range(i+1, len(tp.columns)):
        lst = reduce(lambda x, y: x + y, ind_same.values(), [])
        if all(temp.iloc[:, i] == tp.iloc[:, j]):
            if i not in ind_same.keys():
                if i not in lst:
                    if j >= i: ind_same[i] = [j+1]
                    else: ind_same[i] = [j]
            else:
                if j >= i: ind_same[i].append(j+1)
                else: ind_same[i].append(j)

print "Full_train shape before dropping similar features: " + str(Full_train.shape)
ind_drop = reduce(lambda x, y: x + y, ind_same.values(), [])                 
Full_train.drop(Full_train.columns[ind_drop], inplace=True, axis=1)
print "Full_train shape after: " + str(Full_train.shape)

Full_train shape before dropping similar features: (76020, 369)
Full_train shape after: (76020, 321)


In [90]:
################################
## Outliers
################################
## adding the column TARGET to investigate the rows in Full_train
temp = pd.concat([Full_train, train['TARGET']], axis=1)
## finding out behavior of features' outliers in temp
prop_1_max, prop_1_min, row_ind_max, row_ind_min, feature = [], [], [], [], []
for name in Full_train.columns:
    col_max = temp[temp[name] == max(temp[name])]['TARGET']
    col_min = temp[temp[name] == min(temp[name])]['TARGET']
    val_max, val_min = np.mean(col_max), np.mean(col_min)
    prop_1_max.append(val_max)
    row_ind_max.append(Full_train.index[np.where(temp[name] == max(temp[name]))])
    prop_1_min.append(val_min)
    row_ind_min.append(Full_train.index[np.where(temp[name] == min(temp[name]))])
    feature.append(name)
    
prop_1 = pd.DataFrame({'Max': prop_1_max, 'Min': prop_1_min, 'row_ind_max': row_ind_max, \
                      'row_ind_min': row_ind_min, 'feature': feature})
print "Number of features for which only 1s hit the max: " \
                 + str(len(prop_1[prop_1.Max == 1]['Max']))
print "Number of features for which only 0s hit the max: " \
                 + str(len(prop_1[prop_1.Max == 0]['Max']))
print "Number of features for which at least one 1 hit the min: " \
                 + str(len(prop_1[prop_1.Min != 0]['Min']))
print "Full_train shape: " + str(Full_train.shape)

Number of features for which only 1s hit the max: 11
Number of features for which only 0s hit the max: 240
Number of features for which at least one 1 hit the min: 295
Full_train shape: (76020, 321)


In [91]:
################################
## Max Outliers
################################
## when the TARGET HAS to be 0 and the outlier is isolated
row_out = [len(l) == 1 for l in prop_1.row_ind_max]
isol_out0 = prop_1[(prop_1.Max == 0) & row_out]
feat0 = isol_out0.feature
row_feat_out0 = [x[0] for x in isol_out0.row_ind_max]

new_col_0_out = pd.Series(np.zeros(len(Full_train)), index = Full_train.index)

for row in row_feat_out0:
    if row in new_col_0_out.index:
        new_col_0_out[row] = 1
    else: print "Nope"
print len(new_col_0_out)
#new_col_0_out = pd.Series(new_col_0_out, index = Full_train.index)

for row, col in zip(row_feat_out0, feat0):
    Full_train.loc[row, col] = 0 ## set outliers to zero
    
Full_train = pd.concat([Full_train, pd.DataFrame({'Out_1': new_col_0_out})], axis=1)

## when the TARGET HAS to be 1 and the outlier is isolated
isol_out1 = prop_1[(prop_1.Max == 1) & row_out]
feat1 = isol_out1.feature
row_feat_out1 = [x[0] for x in isol_out1.row_ind_max]

new_col_1_out = pd.Series(np.zeros(len(Full_train)), index = Full_train.index)
for row in row_feat_out1:
    if row in new_col_1_out.index:
        new_col_1_out[row] = 1
    else: print "Nope"
#new_col_1_out = pd.Series(new_col_1_out, index = Full_train.index)

for row, col in zip(row_feat_out1, feat1):
    Full_train.loc[row, col] = 0 ## set outliers to zero
    
Full_train = pd.concat([Full_train, pd.DataFrame({'Out_2': new_col_1_out})], axis=1)
    
Full_train.describe()

76020


,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,...,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,Out_1,Out_2
count,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,...,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000
mean,-1523.202407,33.211484,83.445834,72.193532,119.253063,3.450766,6.327031,0.326127,0.480533,3.052352,...,1.096983,10.389328,7.576777,25.739352,1.534318,67.061907,51.380409,116945.954935,0.001171,0.000066
std,39033.462232,12.954429,1424.005518,336.117157,540.978750,88.238651,148.404448,19.071109,27.573773,90.465520,...,86.721969,599.925263,423.238560,1235.142300,117.681492,3196.360884,2460.740451,164460.581810,0.034196,0.008110
min,-999999.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,23.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,67870.305000,0.000000,0.000000
50%,2.000000,28.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,106408.485000,0.000000,0.000000
75%,2.000000,40.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,118753.320000,0.000000,0.000000
max,235.000000,104.000000,135000.000000,9968.040000,16086.060000,7904.220000,10351.950000,1800.000000,3810.000000,7904.220000,...,11473.050000,63317.190000,42767.160000,126006.180000,18928.590000,397884.300000,374947.530000,13268621.280000,1.000000,1.000000


In [92]:
##################################
# Normalize train (with no target)
##################################
Full_train = min_max_scaler.fit_transform(Full_train)
#Full_train.to_csv("/Users/boulenge/Desktop/Projects/Project4 - ML/Full_train.csv")
train_target = train.iloc[:, 369]

# Normalize test
Full_test = test
Full_test = min_max_scaler.fit_transform(Full_test)
Full_test = pd.DataFrame(Full_test, index = test.index)
#Full_test.to_csv("/Users/boulenge/Desktop/Projects/Project4 - ML/Full_test.csv")

######### Full train ##################
## Slow CV on Full train set
# Cross-Validation and evaluate_model: 75% train - 25% test
X_train, X_test, y_train, y_test = \
             cv.train_test_split(Full_train, train_target, test_size=0.25, random_state=0, \
                                 stratify = train_target)


def evaluate_model(clf):
    """Scores a model using log loss with the created train and test sets."""
    start = time.time()
    clf.fit(X_train, y_train)
    print "Train score:", sklearn.metrics.roc_auc_score(y_train, clf.predict_proba(X_train)[:,1])
    print "Test score:", sklearn.metrics.roc_auc_score(y_test, clf.predict_proba(X_test)[:,1])
    print "Total time:", time.time() - start
    


print "Full Training: " + str(X_train.shape) + str(y_train.shape)
print "Full Test: " + str(X_test.shape) + str(y_test.shape)

Full Training: (57015, 323)(57015,)
Full Test: (19005, 323)(19005,)


In [96]:
################################
## Auto Grid search
################################

#param = {'max_depth': [10, 11, 12], \
#         'n_estimators': [49, 50, 51], \
#         'min_child_weight': [1, 2], \
#         'colsample_bytree': [0.65, 0.7, 0.75],
#         'base_score': [0.25, 0.3, 0.35], \
#         'learning_rate': [0.05, 0.1, 0.15], \
#         'scale_pos_weight': [1],
#         'nthread': [1],
#         'reg_lambda': [0, 1, 5],
#         'scale_pos_weight': [1, 10, 15, 20]
#         }

#param = {'max_depth': [10], \
#         'n_estimators': [50], \
#         'min_child_weight': [1], \
#         'colsample_bytree': np.arange(0.735, 0.745, 0.001),
#         'base_score': np.arange(0.295, 0.305, 0.001), \
#         'learning_rate': np.arange(0.1, 0.115, 0.001), \
#         'scale_pos_weight': np.arange(0.1, 1.5, 0.1),
#         'nthread': [1],
#         'reg_lambda': [1],
#         }

param = {'max_depth': range(9, 11), \
         'n_estimators': [50], \
         'min_child_weight': [1], \
         'colsample_bytree': [0.7],
         'base_score': [0.29, 0.3], \
         'learning_rate': np.arange(0.09, 0.11, 0.01), \
         'scale_pos_weight': [1],
         'nthread': [1],
         'reg_lambda': [1], \
         'reg_alpha': [4], \
         'max_delta_step': np.arange(0, 3, 1)
         }

print str(np.product([len(param[x]) for x in param])) + " models to test"

xgb_mod = XGBClassifier(objective='binary:logistic')
clf = GridSearchCV(xgb_mod, param, verbose=2, scoring='roc_auc', cv=3)
clf.fit(X_train, y_train)

print(clf.best_score_)
print(clf.best_params_)

36 models to test
Fitting 3 folds for each of 36 candidates, totalling 108 fits
[CV] reg_alpha=4, colsample_bytree=0.7, scale_pos_weight=1, learning_rate=0.09, max_delta_step=0, nthread=1, base_score=0.29, n_estimators=50, reg_lambda=1, min_child_weight=1, max_depth=9 
[CV]  reg_alpha=4, colsample_bytree=0.7, scale_pos_weight=1, learning_rate=0.09, max_delta_step=0, nthread=1, base_score=0.29, n_estimators=50, reg_lambda=1, min_child_weight=1, max_depth=9 -  43.5s
[CV] reg_alpha=4, colsample_bytree=0.7, scale_pos_weight=1, learning_rate=0.09, max_delta_step=0, nthread=1, base_score=0.29, n_estimators=50, reg_lambda=1, min_child_weight=1, max_depth=9 
[CV]  reg_alpha=4, colsample_bytree=0.7, scale_pos_weight=1, learning_rate=0.09, max_delta_step=0, nthread=1, base_score=0.29, n_estimators=50, reg_lambda=1, min_child_weight=1, max_depth=9 -  30.0s
[CV] reg_alpha=4, colsample_bytree=0.7, scale_pos_weight=1, learning_rate=0.09, max_delta_step=0, nthread=1, base_score=0.29, n_estimators=50,

[Parallel(n_jobs=1)]: Done  40 tasks       | elapsed: 30.4min


[CV]  reg_alpha=4, colsample_bytree=0.7, scale_pos_weight=1, learning_rate=0.11, max_delta_step=0, nthread=1, base_score=0.29, n_estimators=50, reg_lambda=1, min_child_weight=1, max_depth=10 -  48.8s
[CV] reg_alpha=4, colsample_bytree=0.7, scale_pos_weight=1, learning_rate=0.11, max_delta_step=0, nthread=1, base_score=0.29, n_estimators=50, reg_lambda=1, min_child_weight=1, max_depth=10 
[CV]  reg_alpha=4, colsample_bytree=0.7, scale_pos_weight=1, learning_rate=0.11, max_delta_step=0, nthread=1, base_score=0.29, n_estimators=50, reg_lambda=1, min_child_weight=1, max_depth=10 -  48.8s
[CV] reg_alpha=4, colsample_bytree=0.7, scale_pos_weight=1, learning_rate=0.11, max_delta_step=1, nthread=1, base_score=0.29, n_estimators=50, reg_lambda=1, min_child_weight=1, max_depth=9 
[CV]  reg_alpha=4, colsample_bytree=0.7, scale_pos_weight=1, learning_rate=0.11, max_delta_step=1, nthread=1, base_score=0.29, n_estimators=50, reg_lambda=1, min_child_weight=1, max_depth=9 -  43.6s
[CV] reg_alpha=4, co

[Parallel(n_jobs=1)]: Done 108 out of 108 | elapsed: 70.0min finished


0.834910201782
{'reg_alpha': 4, 'colsample_bytree': 0.7, 'scale_pos_weight': 1, 'learning_rate': 0.10999999999999999, 'max_delta_step': 1, 'nthread': 1, 'base_score': 0.29, 'n_estimators': 50, 'reg_lambda': 1, 'min_child_weight': 1, 'max_depth': 10}
